In [1]:
import numpy as np

import spglib as spg

from hilde.io import read
from hilde.harmonic_analysis import HarmonicAnalysis
from hilde.structure.convert import to_spglib_cell
from hilde.helpers.numerics import clean_matrix
from hilde.helpers.lattice_points import get_commensurate_q_points

from hilde.spglib.q_mesh import get_ir_reciprocal_mesh

In [2]:
mat = 'mgo'
primitive = read(f'{mat}/geometry.in')
supercell = read(f'{mat}/geometry.in.supercell')
spg.get_spacegroup(to_spglib_cell(supercell))

'Fm-3m (225)'

In [3]:
ha = HarmonicAnalysis(primitive, supercell)

Set up harmonic analysis for MgO:
.. found 32 (31) lattice points in 0.002s
.. found 32 (31) lattice points in 0.001s
** Force constants not set, your choice.
.. time elapsed: 0.132s


In [4]:
my_mesh = ha.q_points_frac

In [5]:
n_qmesh = (my_mesh.max(axis=0) - my_mesh.min(axis=0)) + 1
n_qmesh

array([5, 5, 5])

In [6]:
def to_frac_primitive(q, primitve, supercell):
    return clean_matrix(q @ supercell.get_reciprocal_cell() @ primitive.cell.T)

In [7]:
mapping, spg_mesh = spg.get_ir_reciprocal_mesh(n_qmesh * 2, to_spglib_cell(supercell))
for ii, q in enumerate(spg_mesh):
    q1 = to_frac_primitive(q, primitive, supercell)
    q2 = to_frac_primitive(spg_mesh[mapping[ii]], primitive, supercell)
    if np.linalg.norm(q1 - [.5, .5, .5]) < .1:
        print(ii, q1, q2)

111 [0.5 0.5 0.5] [0.5 0.5 0.5]


In [8]:
# create the list of irreducible grid points and the mapping to the reduced list

# irreducible grid points
ir_grid = spg_mesh[np.unique(mapping)]

# dict translates between full and reduced grid point indices
dct = {}
for nn, ii in enumerate(np.unique(mapping)):
    dct[ii] = nn

# mapping to the reduced indices
ir_mapping = np.array([dct[ii] for ii in mapping])

In [9]:
# verify that this mapping works
all((spg_mesh[q] == ir_grid[ir_mapping[i]]).all() for i, q in enumerate(mapping))

True

In [10]:
# match the q points from my list with the ones from spglib

n_lp = len(my_mesh)
match_list = -np.ones(n_lp, dtype=int)
my_mapping = -np.ones(n_lp, dtype=int)

for i1, q1 in enumerate(my_mesh):
    for i2, q2 in enumerate(spg_mesh):
        if np.linalg.norm(q1 - q2) < 1e-12:
            # exchange the q point in spg list with my one
            # ir_grid[ir_mapping[i2]] = q1
            match_list[i1] = i2
            my_mapping[i1] = ir_mapping[i2]

# sanity checks
assert -1 not in match_list, match_list
assert -1 not in my_mapping, my_mapping
assert len(np.unique(match_list)) == n_lp, (len(np.unique(match_list)), match_list)

In [11]:
match_list, np.unique(my_mapping)

(array([  0,   1,  10, 100,  11, 101, 110, 911, 191, 119, 111,   2,  20,
        200,  12,  21, 102, 120, 201, 210, 912, 921, 192, 291, 112, 129,
        121, 219, 211,   3,  30, 300]),
 array([ 0,  1,  2,  3,  6,  7, 21, 22]))

In [12]:
my_ir_grid_temp = clean_matrix(ir_grid[np.unique(my_mapping)] @ supercell.get_reciprocal_cell() @ primitive.cell.T)
my_ir_grid_temp

array([[0.  , 0.  , 0.  ],
       [0.  , 0.25, 0.25],
       [0.  , 0.5 , 0.5 ],
       [0.  , 0.75, 0.75],
       [0.25, 0.25, 0.5 ],
       [0.25, 0.5 , 0.75],
       [0.5 , 0.5 , 0.5 ],
       [0.5 , 0.75, 0.75]])

In [13]:
# wrap atoms back
my_ir_grid_temp_cleaned = clean_matrix((my_ir_grid_temp + .001) % 1 - .001)
my_ir_grid_temp_cleaned

array([[0.  , 0.  , 0.  ],
       [0.  , 0.25, 0.25],
       [0.  , 0.5 , 0.5 ],
       [0.  , 0.75, 0.75],
       [0.25, 0.25, 0.5 ],
       [0.25, 0.5 , 0.75],
       [0.5 , 0.5 , 0.5 ],
       [0.5 , 0.75, 0.75]])

In [14]:
dct = {}
for nn, ii in enumerate(np.unique(my_mapping)):
    dct[ii] = nn
    
my_ir_mapping = np.array([dct[ii] for ii in my_mapping])
my_ir_mapping

array([0, 1, 1, 1, 4, 4, 4, 6, 6, 6, 6, 2, 2, 2, 5, 5, 5, 5, 5, 5, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 3, 3, 3])

In [15]:
my_ir_grid = []
ir_temp_mapping = []
for ii, ir_q_temp in enumerate(my_ir_grid_temp_cleaned):
    for jj, ir_q in enumerate(my_ir_grid):
        if np.linalg.norm(ir_q_temp - ir_q) < 1e-12:
            # they are the same:
            my_ir_mapping[my_ir_mapping == ii] = jj
            break
    else:
        # otherwise add to list
        my_ir_grid.append(ir_q_temp)

In [16]:
# translate back to integers (reference with supercell)
my_ir_grid = np.array(my_ir_grid)
my_ir_grid = clean_matrix(my_ir_grid @ primitive.get_reciprocal_cell() @ supercell.cell.T)
my_ir_grid

array([[0., 0., 0.],
       [1., 0., 0.],
       [2., 0., 0.],
       [3., 0., 0.],
       [1., 1., 0.],
       [2., 1., 0.],
       [1., 1., 1.],
       [2., 1., 1.]])

In [17]:
# verify
# [(my_ir_grid[my_ir_mapping[i]] % n_qmesh, ir_grid[q] % n_qmesh) for i, q in enumerate(my_mapping)]

In [18]:
print(f'Number of q points reduced from\n  {len(my_ir_mapping)} to {len(np.unique(my_ir_mapping))}')

Number of q points reduced from
  32 to 8


In [19]:
# now implemented in HarmonicAnalysis
m, iqs = ha.irreducible_q_points_frac

.. number of q points reduced from 32 to 8 in 0.184s


In [20]:
for fq in iqs:
    cart_q = fq @ supercell.get_reciprocal_cell()
    frac_q = clean_matrix(cart_q @ primitive.cell.T)
    # map back to supercell
    frac_q_supercell = clean_matrix(((frac_q + .01) % 1 - .01) @ primitive.get_reciprocal_cell() @ supercell.cell.T)
    print(fq, frac_q, frac_q_supercell)

[0. 0. 0.] [0. 0. 0.] [0. 0. 0.]
[1. 0. 0.] [0.   0.25 0.25] [1. 0. 0.]
[2. 0. 0.] [0.  0.5 0.5] [2. 0. 0.]
[3. 0. 0.] [0.   0.75 0.75] [3. 0. 0.]
[1. 1. 0.] [0.25 0.25 0.5 ] [1. 1. 0.]
[2. 1. 0.] [0.25 0.5  0.75] [2. 1. 0.]
[1. 1. 1.] [0.5 0.5 0.5] [1. 1. 1.]
[2. 1. 1.] [0.5  0.75 0.75] [2. 1. 1.]
